In [2]:
!pip install vllm==0.6.0


  Using cached vllm-0.6.0-cp38-abi3-manylinux1_x86_64.whl.metadata (2.2 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached transformers-4.50.1-py3-none-any.whl.metadata (39 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached protobuf-6.30.2-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached aiohttp-3.11.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached openai-1.68.2-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached prometheus_fastapi_instrumentator-7

In [1]:
%cd /workspace/LLaMA-Factory

/workspace/LLaMA-Factory


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import string
from vllm import LLM, SamplingParams

# LLM
llm = LLM(model="irene93/Llama-3-Company-Extract")

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

WARNING 03-27 04:13:26 arg_utils.py:862] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 03-27 04:13:26 config.py:999] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 03-27 04:13:26 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='irene93/Llama-3-Company-Extract', speculative_config=None, tokenizer='irene93/Llama-3-Company-Extract', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quan

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-27 04:15:31 model_runner.py:926] Loading model weights took 6.0160 GB
INFO 03-27 04:15:31 gpu_executor.py:122] # GPU blocks: 37314, # CPU blocks: 2340
INFO 03-27 04:15:36 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-27 04:15:36 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-27 04:15:48 model_runner.py:1335] Graph capturing finished in 12 secs.


In [17]:
# 프롬프트 템플릿 준비
template = string.Template("""<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>

\n 뉴스데이터 :${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|> 
""")

system_prompt = '''1. 주어진 입력에 나와있는 회사명을 찾으세요 
2. 회사명을 찾을수 없다면, 빈 파이썬 리스트를 출력하고, 임의로 답변하려고 시도하지 마세요. 
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 뉴스데이터:이 주어지면 회사리스트: 다음에 파이썬 리스트 형태로 작성하세요.'''

user_prompt = '''LG에너지솔루션, 배터리 관리 분야 새 얼굴 '비 어라운드' 만들었다 LG에너지솔루션의 배터리 관리 토털 설루션(BMTS) 브랜드 'B.around'(비.어라운드) 로고. LG에너지솔루션 제공  잇단 화재로 전기차 배터리에 불안과 공포(전기차 포비아)가 확산한 가운데 LG에너지솔루션(LG엔솔)이 배터리 관리 토털 설루션(BMTS) 브랜드를 만들고 사업 확장에 나선다. 전기차 배터리 생산을 넘어 소프트웨어 부문 강화로 수익원을 넓히려는 움직임으로 풀이된다.  LG엔솔은 25일 '항상 고객의 곁에'(Be around your side)란 뜻의 BMTS 브랜드 'B.around'(비.어라운드)를 공개했다. 어떤 상황에서든 배터리 상태를 실시간 모니터링하는 LG엔솔 BMTS의 기술 경쟁력을 상징한다는 설명이다.  비.어라운드 제품에는 기존 배터리 관리 시스템(BMS)에 클라우드와 인공지능(AI) 기술을 결합해 화재방지 등을 위한 안전 진단, 퇴화·수명 예측 등을 더 쉽게 할 계획이다. 배터리 불량 유형을 사전에 진단하고 퇴화 상태를 점검하는 소프트웨어를 통해 오랫동안 쓸 수 있게 할 방침이다.  이달훈 LG에너지솔루션 BMS개발센터장 상무는 "오랜 기간 안전 진단 시스템을 위해 노력했고 이제는 배터리 관리 소프트웨어 사업으로 확장해 배터리의 건강한 사용을 위해 설루션을 제공하는 기업이 되려고 한다"고 말했다.  김청환 기자 chk@hankookilbo.com'''

# 메시지 준비
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# 프롬프트 생성
prompt = template.safe_substitute({
    "system_content": messages[0]["content"],
    "user_content": messages[1]["content"]
})

In [18]:
prompt = prompt.strip() + '회사리스트:' 

In [19]:
print(prompt)

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
1. 주어진 입력에 나와있는 회사명을 찾으세요 
2. 회사명을 찾을수 없다면, 빈 파이썬 리스트를 출력하고, 임의로 답변하려고 시도하지 마세요. 
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 뉴스데이터:이 주어지면 회사리스트: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>


 뉴스데이터 :LG에너지솔루션, 배터리 관리 분야 새 얼굴 '비 어라운드' 만들었다 LG에너지솔루션의 배터리 관리 토털 설루션(BMTS) 브랜드 'B.around'(비.어라운드) 로고. LG에너지솔루션 제공  잇단 화재로 전기차 배터리에 불안과 공포(전기차 포비아)가 확산한 가운데 LG에너지솔루션(LG엔솔)이 배터리 관리 토털 설루션(BMTS) 브랜드를 만들고 사업 확장에 나선다. 전기차 배터리 생산을 넘어 소프트웨어 부문 강화로 수익원을 넓히려는 움직임으로 풀이된다.  LG엔솔은 25일 '항상 고객의 곁에'(Be around your side)란 뜻의 BMTS 브랜드 'B.around'(비.어라운드)를 공개했다. 어떤 상황에서든 배터리 상태를 실시간 모니터링하는 LG엔솔 BMTS의 기술 경쟁력을 상징한다는 설명이다.  비.어라운드 제품에는 기존 배터리 관리 시스템(BMS)에 클라우드와 인공지능(AI) 기술을 결합해 화재방지 등을 위한 안전 진단, 퇴화·수명 예측 등을 더 쉽게 할 계획이다. 배터리 불량 유형을 사전에 진단하고 퇴화 상태를 점검하는 소프트웨어를 통해 오랫동안 쓸 수 있게 할 방침이다.  이달훈 LG에너지솔루션 BMS개발센터장 상무는 "오랜 기간 안전 진단 시스템을 위해 노력했고 이제는 배터리 관리 소프트웨어 사업으로 확장해 배터리의 건강한 사용을 위해 설루션을 제공하는 기업이 되려고 한다"고 말했다.  김청환 기자 chk@hankooki

In [20]:
# 샘플링 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
    stop=["\n<|end_of_text|>", "<|eot_id|>"],  # 문자열로 중지 토큰 지정
    repetition_penalty=1.0
)

# 생성
outputs = llm.generate([prompt], sampling_params)

# 결과 출력
for output in outputs:
    print()
    print(output.outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.42it/s, est. speed input: 2447.92 toks/s, output: 123.94 toks/s]


 ['LG에너지솔루션', 'LG엔솔루션', '배터리 관리 토털 설루션']


In [49]:
import pandas as pd

df = pd.read_json('company_extract_test.json')

In [50]:
inputs = df['instruction'].to_list()

In [51]:
def make_prompt(system_prompt, news_article):
    # 프롬프트 템플릿 준비
    template = string.Template("""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
${system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>
    
${user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")
    
    user_prompt = news_article
    
    # 메시지 준비
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # 프롬프트 생성
    prompt = template.safe_substitute({
        "system_content": messages[0]["content"],
        "user_content": messages[1]["content"]
    })

    prompt = prompt.strip() + "회사리스트:"
    return prompt

In [52]:
prompt = make_prompt(system_prompt, inputs[0])
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
1. 주어진 입력에 나와있는 회사명을 찾으세요 
2. 회사명을 찾을수 없다면, 빈 파이썬 리스트를 출력하고, 임의로 답변하려고 시도하지 마세요. 
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 뉴스데이터:이 주어지면 회사리스트: 다음에 파이썬 리스트 형태로 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>
    
뉴스데이터: 5일 오전 비트코인이 2만달러를 회복했다. 사진 이미지투데이 비트코인이 반등세를 보이며 2만달러를 회복했다. 5일 오전 7시 49분 글로벌 코인시황 중계 사이트 코인마켓캡에서 비트코인 1BTC 비트코인 단위 당 가격은 24시간 전 대비 3.77% 상승한 2만1달러에 거래되고 있다. 이더리움은 1ETH 이더리움 단위 당 6.36% 오른 1140달러 솔라나는 1SOL 솔라나 단위 당 9.38% 뛴 36.38달러에 거래되고 있다. 리플은 1XRP 리플 단위 당 0.32달러에 거래되며 2.35% 상승했다. 투자자들이 이번 주 발표될 미국의 6월 고용지표 중앙은행인 연방준비제도 Fed·연준 의 6월 연방공개시장위원회 FOMC 회의록 공개를 주시하고 있는 가운데 비트코인 바닥론 이 슬며시 고개를 들고 있다. 가상화폐 암호화폐 전문 매체 유투데이에 따르면 알리 마르티네즈 애널리스트는 최근 트위터를 통해 가상화폐 온체인 데이터 분석 플랫폼 샌티멘트의 비트코인 MVRV 365D 지표가 2015년 1월 기록한 56.85% 2018년 12월 기록한 55.62%에 가까워졌다 며 지난달 중순 50.09% 현재 48.23%를 나타내고 있는 해당 지표는 지난 약세 사이클 바닥에서 나타났던 수치와 비슷한 수준을 기록하고 있다 고 말했다. 투자 심리는 여전히 얼어붙어 있으나 소폭 개선됐다. 국내 가상자산 거래소 업비트를 운영하고 있는 두나무의 디지털자산 공포 탐욕지수 는 전일 기준 32

In [53]:
prompts = []
for input in inputs:
    prompts.append(make_prompt(system_prompt, input))

In [54]:
results = llm.generate(prompts[:5], sampling_params)

Processed prompts: 100%|██████████| 5/5 [00:00<00:00, 14.32it/s, est. speed input: 9765.93 toks/s, output: 315.57 toks/s]


In [55]:
for result in results:
    print(result.outputs[0].text)
    print('--')

 ['비트코인', '이더리움', '솔라나']
--
 ['쌍용자동차', 'KG그룹', '영종도 네스트 호텔']
--
 ['농림축산식품부', '대형마트', '국적운송사']
--
 ['피토틱스', 'FCCE', '피토틱스 메타바이오틱스']
--
 ['BMW', '메르세데스벤츠', '폭스바겐']
--


In [56]:
results = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 500/500 [00:14<00:00, 34.41it/s, est. speed input: 25642.66 toks/s, output: 611.92 toks/s]


In [57]:
import ast
import re

In [66]:

preds = []
for pred in results:
    preds.append(ast.literal_eval(pred.outputs[0].text.strip()))

In [67]:
preds[1]

['쌍용자동차', 'KG그룹', '영종도 네스트 호텔']

In [68]:
labels = []
for label in df['output'].to_list():
    labels.append(ast.literal_eval(re.sub('회사리스트 ?: ?','',label).strip()))

In [69]:
labels[1]

['쌍용자동차', 'KG그룹']

In [71]:
from sklearn.metrics import f1_score, accuracy_score
from itertools import chain

def calculate_metrics(labels, preds):
    """
    Calculate accuracy and F1 score for list of lists of companies.

    :param labels: List of lists of true company labels.
    :param preds: List of lists of predicted company labels.
    :return: Dictionary containing accuracy and F1 score.
    """
    if len(labels) != len(preds):
        raise ValueError("Labels and predictions must have the same number of instances.")
    
    # Flatten lists for binary representation
    all_companies = set(chain.from_iterable(labels + preds))
    all_companies = list(all_companies)  # Ensure consistent ordering
    
    # Create binary vectors for each instance
    y_true = []
    y_pred = []
    for label, pred in zip(labels, preds):
        label_set = set(label)
        pred_set = set(pred)
        
        y_true.append([1 if company in label_set else 0 for company in all_companies])
        y_pred.append([1 if company in pred_set else 0 for company in all_companies])
    
    # Flatten binary vectors for overall metrics
    y_true_flat = list(chain.from_iterable(y_true))
    y_pred_flat = list(chain.from_iterable(y_pred))
    
    # Calculate metrics
    accuracy = accuracy_score(y_true_flat, y_pred_flat)
    f1 = f1_score(y_true_flat, y_pred_flat, average="micro")  # Use 'micro' for multilabel cases
    
    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

# Example usage
metrics = calculate_metrics(labels, preds)
print("Metrics:", metrics)


Metrics: {'accuracy': 0.9980953800298062, 'f1_score': 0.9980953800298062}
